<a href="https://colab.research.google.com/github/SeanSDarcy2001/friedBrains/blob/main/friedBrainsModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports

In [1]:
from sklearn.datasets import make_classification, make_regression
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


data simulation

In [82]:
#for sober/intoxicated classifier

features, output = make_classification(n_samples = 1000,
                                       n_features = 3,
                                       n_informative= 3,
                                       n_redundant=0,
                                       n_classes = 2,
                                       weights = [0.75, 0.25])

#weights are for class generation. Roughly expect to play sober 90% of the time and intoxicated the other 10%

print(pd.DataFrame(features, columns=['Reaction Time', 'Accuracy', 'Literally Anything Else']).head())
print(pd.DataFrame(output, columns=['Class']))



   Reaction Time  Accuracy  Literally Anything Else
0       1.253769  0.906094                 1.945247
1      -0.649110 -1.528401                -0.384830
2       1.675395  0.248165                 1.314595
3      -1.336068 -2.152434                -1.735630
4      -0.028392  1.538973                 2.527456
     Class
0        0
1        0
2        0
3        0
4        1
..     ...
995      0
996      0
997      0
998      1
999      0

[1000 rows x 1 columns]


In [86]:
#mlp

class MLP(nn.Module) :
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3, 9, bias=True)
        self.h1 = nn.Linear(9, 3, bias=True)
        self.h2 = nn.Linear(3, 2, bias=True)

    def forward(self, x):
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.h1(x))
        x = torch.sigmoid(self.h2(x))
        return x

In [90]:
#!pip install skorch
from skorch import NeuralNetClassifier


    
features = features.astype(np.float32)
output = output.astype(np.int64)


trainFraction = .8
sample = np.random.uniform(size = 1000) < trainFraction
xtrain = features[sample]
ytrain = output[sample]
xtest = features[~sample]
ytest = output[~sample]

#ytrain = torch.tensor(ytrain).float()
#ytest = torch.tensor(ytest).float()

xtorch = torch.from_numpy(xtrain).float()
ytorch = torch.from_numpy(ytrain).float().unsqueeze(1)

learning_rate = 1.25
model = MLP()

def init_weights(m):
  if isinstance(m, nn.Linear):
    torch.nn.init.xavier_uniform(m.weight)
    m.bias.data.fill_(0.01)

model.apply(init_weights)

net = NeuralNetClassifier(model, max_epochs = 100, lr = learning_rate, iterator_train__shuffle = True)
net.fit(xtrain, ytrain)

preds = net.predict(xtest)
acc = 0
tot = 0
for i in range(len(preds)) :
  if preds[i] == ytest[i] :
    acc += 1
    tot += 1
  else :
    tot += 1
print("MLP Test Accuracy:", acc/tot)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.3558       0.7484        0.1382  0.0112
      2        0.1045       0.7484        0.0696  0.0120
      3        0.0580       0.7484        0.0445  0.0120
      4        0.0389       0.7484        0.0320  0.0113
      5        0.0288       0.7484        0.0248  0.0116
      6        0.0227       0.7484        0.0200  0.0119
      7        0.0186       0.7484        0.0168  0.0116
      8        0.0157       0.7484        0.0144  0.0119
      9        0.0136       0.7484        0.0126  0.0117
     10        0.0119       0.7484        0.0111  0.0120
     11        0.0106       0.7484        0.0100  0.0121
     12        0.0095       0.7484        0.0090  0.0128
     13        0.0087       0.7484        0.0082  0.0106


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


     14        0.0079       0.7484        0.0076  0.0203
     15        0.0073       0.7484        0.0070  0.0117
     16        0.0068       0.7484        0.0065  0.0166
     17        0.0063       0.7484        0.0061  0.0109
     18        0.0059       0.7484        0.0057  0.0139
     19        0.0055       0.7484        0.0054  0.0129
     20        0.0052       0.7484        0.0051  0.0137
     21        0.0049       0.7484        0.0048  0.0135
     22        0.0047       0.7484        0.0046  0.0143
     23        0.0044       0.7484        0.0043  0.0142
     24        0.0042       0.7484        0.0041  0.0152
     25        0.0040       0.7484        0.0040  0.0157
     26        0.0039       0.7484        0.0038  0.0136
     27        0.0037       0.7484        0.0036  0.0114
     28        0.0035       0.7484        0.0035  0.0125
     29        0.0034       0.7484        0.0034  0.0124
     30        0.0033       0.7484        0.0032  0.0134
     31        0.0032       0.7

In [93]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()

logReg.fit(xtrain, ytrain)
print("Logistic Regression Test Accuracy:", logReg.score(xtest, ytest))

Logistic Regression Test Accuracy: 0.775609756097561
